In [4]:
import numpy as np
import json
from scipy.spatial import distance_matrix
import matplotlib.pyplot as plt
import sys
sys.path.append('C:\\\\Users\\\\dell\\\\Desktop\\\\phasemapy')
import os
from copy import deepcopy
from scipy.constants import h, c, e
from pymatgen.core import Element
from monty.json import MontyDecoder, MontyEncoder
from pymatgen.analysis.diffraction.xrd import XRDCalculator
import numpy as np
from phasemapy.dataio import InstanceData
from phasemapy.parser import ICDDEntry,ICDDEntriesBiCuV, ICDDEntryPreprocessor
from phasemapy.solver import Phase, Sample
import matplotlib.pyplot as plt
chemsys = ['Bi', 'Cu', 'V']
oxide_system = True
photon_e = 13e3
max_q_shift = 0.05
resample_density = 1000
initial_alphagamma = 0.1
SUM_NORM = 6000
loss_weight = {'xrd_loss': 6.0, 'comp_loss': 2.0, 'entropy_loss': 0.01}

In [5]:
#load entry pool: 100 ICDD entries
with open('C:/Users/dell/Desktop/phasemapy/scripts_Bi_Cu_V_O/data/ICDD_entries_Bi_Cu_V.json') as f:
    entries_Bi_Cu_V = json.load(f, cls=MontyDecoder)
with open('C:/Users/dell/Desktop/phasemapy/scripts_Bi_Cu_V_O/data/Phase_update_merge_by_xrd.json') as f:
    Phase_update = json.load(f, cls=MontyDecoder)

entries = [ICDDEntriesBiCuV.from_ICDD_Bi_Cu_V(entries_Bi_Cu_V[i]) for i in Phase_update]

In [6]:
#load instance data:307 instances
instance_data = InstanceData.from_file_BiCuV('C:/Users/dell/Desktop/phasemapy/scripts_Bi_Cu_V_O/data/Instance_data_Bi_Cu_V', chemsys, photon_e)

In [8]:
comp_dist = distance_matrix(instance_data.sample_comp, instance_data.sample_comp)
nn_list = {i: np.where((comp_dist[i] < 0.15) & (comp_dist[i] > 0))[0] for i in range(instance_data.sample_num)}
samples = []

for i in range(instance_data.sample_num):
    solution_file = f'solution/samples{i}.json'
    if os.path.exists(solution_file):
        with open(solution_file) as f:
            sample = json.load(f, cls=MontyDecoder)
            samples.append(sample)

In [69]:
new_samples = deepcopy(samples)

In [70]:
for sample in new_samples[168:]:
    if True:
        # if min([count_act[_.entry_id] for _ in sample.entries])<5.0:
        if len(sample.phase_fractions)==0:
            sample.solution = new_samples[sample.sample_id-1].solution
        if sample.R > 0.4 and len(sample.phase_fractions)>0:
#         print(sample.sample_id, sample.loss(loss_weight))
            candidate_entries = []
            for i in nn_list[sample.sample_id]:
                candidate_entries += samples[i].entries
            candidate_entries = list(set(candidate_entries))
            solution = []
            for e in candidate_entries:
                phase = Phase.from_entry_and_instance_data(e, 1 / len(candidate_entries), instance_data)
                solution.append(phase)

            new_sample = deepcopy(sample)
            new_sample.solution = solution            
            new_sample.refine_one_by_one()
            new_sample.refine_all_fractions()
#             new_sample.print_solution()
            new_sample.prune_candidate_based_on_xrd(plot=False, cutoff=0.001)
            new_sample.update_solution_new(0.03,0.2999,new_sample.max_q_shift)
            new_sample.refine_one_by_one()
            new_sample.refine_all_fractions()
#             new_sample.print_solution()
#             new_sample.plot(perphase=True)
            # au = new_sample.to_auto
            new_sample = new_sample.optimize(num_epoch=500, print_prog=True, loss_weight=loss_weight)
            new_sample.update_solution_new(0.03, 0.2999, new_sample.max_q_shift)

            new_sample = new_sample.optimize(num_epoch=3000, print_prog=True, loss_weight=loss_weight)
            new_sample.update_solution_new(0.01, 0.2999, new_sample.max_q_shift)

            new_sample.refine_one_by_one()
            new_sample.refine_all_fractions()

            new_sample.update_solution_new(0.01, 0.2999, new_sample.max_q_shift)
            new_sample.refine_one_by_one()
            new_sample.refine_all_fractions()
#             new_sample.print_solution()
#             new_sample.plot(perphase=True)
#             new_samples[sample.sample_id] = new_sample
            samples[sample.sample_id] = new_sample

            print(new_sample.loss(loss_weight))
            if new_sample.loss(loss_weight) <= sample.loss(loss_weight):
#                 sample.print_solution()
#                 new_sample.print_solution()
                samples[sample.sample_id] = new_sample
                solution_file = f'solution/samples{sample.sample_id}.json'
                with open(solution_file, 'w') as f:
                    json.dump( samples[sample.sample_id], f, cls=MontyEncoder)

In [71]:
new_samples = []

for i in range(instance_data.sample_num):
    solution_file = f'solution/samples{i}.json'
    if os.path.exists(solution_file):
        with open(solution_file) as f:
            sample = json.load(f, cls=MontyDecoder)
            new_samples.append(sample)

for sample in new_samples:
    sample.plot(perphase=True, saveplot=f'solution_figures/sample_{sample.sample_id}.pdf')